In [1]:
from tst.domutil.util import *
from tst.domutil.test import *

/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)
/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/subprocess32.py:472: RuntimeWarning: The _posixsubprocess module is not being used. Child process reliability may suffer if your program uses threads.
  "program uses threads.", RuntimeWarning)


In [7]:
import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *
reload(sys.modules["tst.domutil.test"])
from tst.domutil.test import *

In [2]:
%%time

hmms = HMMprofile.objects.defer('text').all().prefetch_related('hits')
hmms = hmms.annotate(hcount = Count('hits')).order_by('id')

hmmids = list(hmms.values_list('id',flat = True))
hcounts = list(hmms.values_list('hcount',flat = True))
sfs = list(hmms.values_list('cath_node__parent',flat = True))



###### Specify the sequence database to work with
sDB = seqDB.objects.get(name = 'CATH')
# sDB.sequence_set.count()
seq = sDB.sequence_set.first()

# seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set')

# seq.hmmprofile_set.count()

hmmids_pad = hmmids[:]
hmmids_pad.insert(0, None)

hcounts_pad = hcounts[:]
hcounts_pad.insert(0,None)

sfs_pad = sfs[:]
sfs_pad.insert(0,None)


CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 38.9 s


In [3]:
%%time 
D_raw  = pk_load('D_raw')
D_norm = pk_load('D_norm')

CPU times: user 23.3 s, sys: 356 ms, total: 23.6 s
Wall time: 23.7 s


In [4]:
test__raw(D_raw,hmms)
test__norm( D_norm, hmms, ISS_normalise_new )

[OK] HMM for 3.20.20.70.102.0  against HMM for 3.40.50.80.20.0  overlaps 1, with 1 from 
0 of -1
[OK] HMM for 3.40.50.720.379.0  against HMM for 3.40.50.720.491.0  overlaps 3, with 3 from 
[OK] HMM for 3.40.50.300.138.0  against HMM for 3.40.50.2300.249.0  overlaps 9, with 9 from 
[OK] HMM for 3.40.190.10.284.0  against HMM for 3.40.190.10.186.0  overlaps 1, with 1 from 
[OK] HMM for 3.40.50.1820.101.0  against HMM for 3.90.1820.10.1.0  overlaps 1, with 1 from 
[OK] HMM for 3.20.20.70.102.0  against HMM for 3.40.50.80.20.0  overlaps:: Expected:0.121225155257, Actual:0.121225155257
0 of -1
[OK] HMM for 3.40.50.720.379.0  against HMM for 3.40.50.720.491.0  overlaps:: Expected:0.227541855883, Actual:0.227541855883
[OK] HMM for 3.40.50.300.138.0  against HMM for 3.40.50.2300.249.0  overlaps:: Expected:0.388674527682, Actual:0.388674527682
[OK] HMM for 3.40.190.10.284.0  against HMM for 3.40.190.10.186.0  overlaps:: Expected:0.138225282556, Actual:0.138225282556
[OK] HMM for 3.40.50.1820.10

In [5]:
%%time
sorted_D_norm = sort_coo( D_norm.tocoo(), order = -1)

CPU times: user 20.2 s, sys: 356 ms, total: 20.6 s
Wall time: 20.5 s


In [12]:
it = izip(
sorted_D_norm,
# sfbools,    
)

CPU times: user 16.5 s, sys: 348 ms, total: 16.8 s
Wall time: 16.7 s


In [26]:
it = (x for x in sorted_D_norm)
D_curr = D_raw.todok()

In [6]:
i = 0
c = counter([],INF = 1, per = 10 )
for (hmmid1,hmmid2,v) in it:
# for (hmmid1,hmmid2,v),sfbool in it:
    c.count()   
#     if sfbool:
#         continue
    
#     print v
    raw = D_curr[hmmid1,hmmid2]
    hmmid1 = int(hmmid1)
    hmmid2 = int(hmmid2)
    hmm1 = hmms.get(id = hmmid1 )
    hmm2 = hmms.get(id = hmmid2 )
#     raise
#     if hmm1.cath_node.parent == hmm2.cath_node.parent:
    cnode1 = hmm1.cath_node
    cnode2 = hmm2.cath_node
#     if cnode1.Class == cnode2.Class:
#     if cnode1.topo == cnode2.topo:
    
    if cnode1.homsf == cnode2.homsf:
        continue
#     if cnode1.superfamily() in ['3.30.200.20', '1.10.10.530']:
#         continue
    
#     if c.i >= 2:
    i+=1
    print hmm1
    print hmm2
    print "norm ISS is %s"%v
    print "raw ISS is %d"%raw
    print hmm1.hits.count() + hmm2.hits.count()
    
    print "http://xhits.cathdb.info/crosshits.php?sf2=%s&sf1=%s&db_source=Crosshits_v4_1_0"%(cnode1.superfamily(),cnode2.superfamily())
    if i == 10:
        break

NameError: name 'it' is not defined

In [1]:
from IPython.display import HTML

CChit = hit4cath2cath.objects.first()
HTML(str(CChit))

In [6]:
idlst = hit4cath2cath.objects.values_list('id', flat = 1 )
print max(idlst) - min(idlst)
print len(idlst)

1272541
1272542


{'ISS_norm': 0.506703406657695, '_state': <django.db.models.base.ModelState object at 0x7f9d1f0b8b90>, 'seqDB_id': 1L, 'ISS_raw': 27L, 'id': 337001L, 'node2_id': 294989L, 'node1_id': 290430L}
